<a href="https://colab.research.google.com/github/eohta/udemy-numpyro-basic/blob/main/08_porkbuns/02_bayes_decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 【ベイズ決定】肉まんの販売個数

モデルから予測された事後予測分布を使って、蒸し器に投入する最適な肉まんの個数を予測させる。

## Package Installation

In [ ]:
!pip install numpyro

インストール完了後にランタイムを再スタートして下さい！

## Import Package

In [ ]:
import numpyro
import numpyro.distributions as dist

import jax
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
numpyro.set_platform('cpu')
numpyro.set_host_device_count(4)

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [10, 4]

## Load & Preprocess Data

In [ ]:
data = pd.DataFrame({
    
    'temperature':[13.2,  2.7,  8.7,  4.5, 10.8, 13.8,  6.2,  5.8,  2.5,  7. , 13.2,
        2.7,  8.7,  4.5, 10.8, 13.8,  6.2,  5.8,  2.5,  7. , 13.2,  2.7,
        8.7,  4.5, 10.8, 13.8,  6.2,  5.8,  2.5,  7. , 13.2,  2.7,  8.7,
        4.5, 10.8, 13.8,  6.2,  5.8,  2.5,  7. , 13.2,  2.7,  8.7,  4.5,
       10.8, 13.8,  6.2,  5.8,  2.5,  7. , 13.2,  2.7,  8.7,  4.5, 10.8,
       13.8,  6.2,  5.8,  2.5,  7. , 13.2,  2.7,  8.7,  4.5, 10.8, 13.8,
        6.2,  5.8,  2.5,  7. , 13.2,  2.7,  8.7,  4.5, 10.8, 13.8,  6.2,
        5.8,  2.5,  7. , 13.2,  2.7,  8.7,  4.5, 10.8, 13.8,  6.2,  5.8,
        2.5,  7. , 13.2,  2.7,  8.7,  4.5, 10.8, 13.8,  6.2,  5.8,  2.5,
        7. ],
    
    'num_sold':[0, 5, 2, 1, 0, 0, 3, 4, 5, 3, 0, 1, 1, 2, 3, 2, 3, 3, 2, 0, 1, 1,
       0, 0, 1, 0, 3, 0, 2, 2, 2, 3, 5, 2, 2, 2, 3, 1, 4, 3, 2, 6, 1, 4,
       4, 1, 5, 6, 4, 2, 3, 3, 0, 3, 1, 1, 2, 4, 3, 1, 1, 3, 1, 1, 3, 2,
       7, 3, 5, 3, 6, 3, 3, 3, 1, 0, 5, 1, 2, 4, 0, 8, 2, 5, 4, 3, 7, 8,
       7, 3, 2, 3, 0, 2, 3, 1, 1, 1, 3, 1],
    
    'store_id':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
})

In [ ]:
x = data['temperature'].values
y = data['num_sold'].values

store_id = data['store_id'].values

num_stores = np.max(store_id) + 1

In [ ]:
x_mu = np.mean(x)
x_sd = np.std(x)

x_scaled = (x - x_mu) / x_sd

## Define Model

In [ ]:
def model(x_scaled=None, store_id=None, y=None, num_data=0):
    
    a = numpyro.sample('a', dist.Normal(0, 10))
    b = numpyro.sample('b', dist.Normal(0, 10))
    
    s = numpyro.sample('s', dist.HalfCauchy(5))
    r = numpyro.sample('r', dist.Normal(0, s), sample_shape=(num_stores,))
    
    theta = a * x_scaled + r[store_id] + b
    
    mu = jax.numpy.exp(theta)
        
    with numpyro.plate('data', num_data):
        
        numpyro.sample('obs', dist.Poisson(mu), obs=y)
    

In [ ]:
nuts = numpyro.infer.NUTS(model)
mcmc = numpyro.infer.MCMC(nuts, num_warmup=500, num_samples=3000, num_chains=4)

mcmc.run(jax.random.PRNGKey(0), x_scaled=x_scaled, store_id=store_id, y=y, num_data=len(y))
mcmc_samples = mcmc.get_samples()

idata = az.from_numpyro(mcmc)

## Bayes Decision

In [ ]:
x_new = np.array([5])

x_scaled_new = (x_new - x_mu) / x_sd

In [ ]:
store_id_new = np.array([6])

In [ ]:
predictive = numpyro.infer.Predictive(model, mcmc_samples)

ppc_samples = predictive(jax.random.PRNGKey(1), x_scaled=x_scaled_new, store_id=store_id_new, num_data=len(x_scaled_new))

idata_ppc = az.from_numpyro(posterior_predictive=ppc_samples)

In [ ]:
num_sales_pred = np.array(ppc_samples['obs'])
num_sales_pred

In [ ]:
num_sales_pred.shape

In [ ]:
az.plot_dist(num_sales_pred)

plt.xlabel('Number of Sales')
plt.ylabel('Probability');

In [ ]:
def estimate_profit(num_sales_pred, num_steamed):
    
    profit = np.minimum(num_sales_pred, num_steamed) * 150 - 50 * num_steamed
    
    return np.mean(profit)

In [ ]:
num_porkbuns_to_steam = np.arange(10)

profit_estimated = [estimate_profit(num_sales_pred, k) for k in num_porkbuns_to_steam]

for k in num_porkbuns_to_steam:

    print('肉まんの個数: {}, 利益: {:3.0f}円'.format(k, np.round(profit_estimated[k])))

In [ ]:
plt.plot(profit_estimated, 'o-')
plt.xticks(np.arange(len(profit_estimated)))

plt.xlabel('Number to Steam')
plt.ylabel('Expected Profit');

## Combine Weather Prediction

In [ ]:
x_new_mu = 5
x_new_sd = 1

In [ ]:
x_new = np.random.normal(loc=x_new_mu, scale=x_new_sd, size=1000)

In [ ]:
fig = plt.subplots(figsize=(10, 4))

sns.histplot(x_new);

In [ ]:
x_scaled_new = (x_new - x_mu) / x_sd

store_id_dup = np.ones_like(x_new, dtype=int) * store_id_new

In [ ]:
predictive = numpyro.infer.Predictive(model, mcmc_samples)

ppc_samples = predictive(jax.random.PRNGKey(1), x_scaled=x_scaled_new, store_id=store_id_dup, num_data=len(x_scaled_new))

idata_ppc = az.from_numpyro(posterior_predictive=ppc_samples)

In [ ]:
num_sales_pred = np.array(ppc_samples['obs'])
num_sales_pred

In [ ]:
num_sales_pred.shape

In [ ]:
az.plot_dist(num_sales_pred)

plt.xlabel('Number of Sales')
plt.ylabel('Probability');

In [ ]:
num_porkbuns_to_steam = np.arange(10)

profit_estimated = [estimate_profit(num_sales_pred, k) for k in num_porkbuns_to_steam]

for k in num_porkbuns_to_steam:

    print('肉まんの個数: {}, 利益: {:3.0f}円'.format(k, np.round(profit_estimated[k])))

In [ ]:
plt.plot(profit_estimated, 'o-')
plt.xticks(np.arange(len(profit_estimated)))

plt.xlabel('Number to Steam')
plt.ylabel('Expected Profit');